Install + Import Libraries


In [ ]:
!pip install xgboost scikit-learn matplotlib pandas seaborn


Load the dataset

Replace 'your_data.csv' with the cleaned + labeled dataset your team will provide

In [ ]:
df = pd.read_csv("your_data.csv")


Inspect structure

In [ ]:
print(df.head())
print(df.info())


Separate Features (X) and Labels (y)
Assume 'label' column exists (1 = failure, 0 = healthy)

In [ ]:
X = df.drop(columns=['label', 'time'])  # drop label + timestamp
y = df['label']


Train-test split (time-based split is even better for PdM, but this is standard)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

Train XGBoost Model

In [ ]:
model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]), # handle imbalance
    use_label_encoder=False,
    eval_metric="logloss")

model.fit(X_train, y_train)

Evaluate Model

In [ ]:
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("ROC-AUC:", roc_auc_score(y_test, y_proba))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Healthy", "Failure"], yticklabels=["Healthy", "Failure"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# ROC Curve
RocCurveDisplay.from_estimator(model, X_test, y_test)
plt.show()

Feature Importance

In [ ]:
plot_importance(model, max_num_features=15, importance_type='weight')
plt.show()